In [1]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model, load_model, save_model

Using TensorFlow backend.


In [4]:
#Resizing images, if needed
SIZE_X = 224
SIZE_Y = 224
n_classes=3 #Number of classes for segmentation

In [5]:
#Capture training image info as a list
train_images = []

In [6]:
for directory_path in glob.glob("/home/gxt/study/LungSegmentation/data/multi-class/Imgs"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path, 0)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        train_images.append(img)

In [7]:
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

In [8]:
print(len(train_images))

98


In [9]:
train_masks = [] 
for directory_path in glob.glob("/home/gxt/study/LungSegmentation/data/multi-class/labels"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        #print(mask_path)
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        train_masks.append(mask)

In [10]:
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

In [ ]:
np.unique(train_masks)

In [ ]:
print(train_masks.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

In [14]:
train_images = np.expand_dims(train_images, axis=3)
train_images = normalize(train_images, axis=1)

train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

In [ ]:
#Create a subset of data for quick testing
#Picking 10% for testing and remaining for training
from sklearn.model_selection import train_test_split
X1, X_test, y1, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)

#Further split training data t a smaller subset for quick testing of models
#X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.2, random_state = 0)

print("Class values in the dataset are ... ", np.unique(y1))  # 0 is the background/few unlabeled 

from keras.utils import to_categorical
#train_masks_cat = to_categorical(y_train, num_classes=n_classes)
#y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))
train_masks_cat = to_categorical(y1, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y1.shape[0], y1.shape[1], y1.shape[2], n_classes))


test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))
print(X1.shape)

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, add, concatenate,UpSampling2D,ZeroPadding2D

In [17]:
def conv_block(inputs, 
            neuron_num, 
            kernel_size,  
            use_bias, 
            padding= 'same',
            strides= (1, 1),
            with_conv_short_cut = False):
        conv1 = Conv2D(
            neuron_num,
            kernel_size = kernel_size,
            activation= 'relu',
            strides= strides,
            use_bias= use_bias,
            padding= padding
        )(inputs)
        conv1 = BatchNormalization(axis = -1)(conv1)

        conv2 = Conv2D(
            neuron_num,
            kernel_size= kernel_size,
            activation= 'relu',
            use_bias= use_bias,
            padding= padding)(conv1)
        conv2 = BatchNormalization(axis = -1)(conv2)

        if with_conv_short_cut:
            inputs = Conv2D(
                neuron_num, 
                kernel_size= kernel_size,
                strides= strides,
                use_bias= use_bias,
                padding= padding
                )(inputs)
            return add([inputs, conv2])

        else:
            return add([inputs, conv2])


def unet_res(n_classes=3, IMG_HEIGHT=352, IMG_WIDTH=352, IMG_CHANNELS=1):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    
    #x = ZeroPadding2D((3, 3))(inputs)
    #x = Input(shape= input_size)
    #print("x=",x.shape)
    # Define the converlutional block 1
    
    x = BatchNormalization(axis=-1)(inputs)
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2),padding='valid')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    print("x=",x.shape)
    x1 = MaxPooling2D(pool_size=(3,3),strides=(2, 2), padding='same')(x)
    print("x1=",x1.shape)
    
    # Define the converlutional block 2
    x2 = conv_block(x1, neuron_num= 64, kernel_size= (3, 3), use_bias= True)
    x2 = conv_block(x2, neuron_num= 64, kernel_size= (3, 3), use_bias= True)
    x2 = conv_block(x2, neuron_num= 64, kernel_size= (3, 3), use_bias= True)
    print("x2=",x2.shape)

    # Define the converlutional block 3
    x3 = conv_block(x2, neuron_num= 128, kernel_size= (3, 3), use_bias= True, strides= (2, 2), with_conv_short_cut= True)
    x3 = conv_block(x3, neuron_num= 128, kernel_size= (3, 3), use_bias= True)
    x3 = conv_block(x3, neuron_num= 128, kernel_size= (3, 3), use_bias= True)
    x3 = conv_block(x3, neuron_num= 128, kernel_size= (3, 3), use_bias= True)
    print("x3=",x3.shape)
    
    # Define the converlutional block 4
    x4 = conv_block(x3, neuron_num= 256, kernel_size= (3, 3), use_bias= True, strides= (2, 2), with_conv_short_cut= True)
    x4 = conv_block(x4, neuron_num= 256, kernel_size= (3, 3), use_bias= True)
    x4 = conv_block(x4, neuron_num= 256, kernel_size= (3, 3), use_bias= True)
    x4 = conv_block(x4, neuron_num= 256, kernel_size= (3, 3), use_bias= True)
    x4 = conv_block(x4, neuron_num= 256, kernel_size= (3, 3), use_bias= True)
    x4 = conv_block(x4, neuron_num= 256, kernel_size= (3, 3), use_bias= True)
    print("x4=",x4.shape)
    
    # Define the converltional block 5
    x5 = conv_block(x4, neuron_num= 512, kernel_size= (3, 3), use_bias= True, strides= (2, 2), with_conv_short_cut= True)
    x5 = conv_block(x5, neuron_num= 512, kernel_size= (3, 3), use_bias= True)
    x5 = conv_block(x5, neuron_num= 512, kernel_size= (3, 3), use_bias= True)
    print("x5=",x5.shape)
    #Define upsample
    up6 = Conv2DTranspose(256,(2,2),strides=(2,2),padding='same')(x5)
    print("up6=",up6.shape)
    #up6 = Conv2D(256, 2, activation='relu', padding = 'same')(UpSampling2D(size=(2,2))(x5))
    merge6 = concatenate([x4,up6],axis = 3)
    print("merge6=[x4,up6]",merge6.shape)
    conv6 = Conv2D(256,3,activation='relu', padding='same')(merge6)
    conv6 = Conv2D(256,3,activation='relu', padding='same')(conv6)
    print("conv6=",conv6.shape)
    
    up7 = Conv2DTranspose(128,(2,2),strides=(2,2),padding='same')(conv6)
    print("up7=",up7.shape)
    #up7 = Conv2D(128, 2, activation='relu', padding = 'same')(UpSampling2D(size=(2,2))(conv6))
    merge7 = concatenate([x3,up7],axis = 3)
    print("merge7=[x3,up7]",merge7.shape)
    conv7 = Conv2D(128,3,activation='relu', padding='same')(merge7)
    conv7 = Conv2D(128,3,activation='relu', padding='same')(conv7)
    print("conv7=",conv7.shape)
    
    up8 = Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(conv7)
    print("up8=",up8.shape)
    #up8 = Conv2D(64, 2, activation='relu', padding = 'same')(UpSampling2D(size=(2,2))(conv7))
    merge8 = concatenate([x2,up8],axis = 3)
    print("merge8=[x2,up8]",merge8.shape)
    conv8 = Conv2D(64,3,activation='relu', padding='same')(merge8)
    conv8 = Conv2D(64,3,activation='relu', padding='same')(conv8)
    print("conv8=",conv8.shape)
    
    up9 =  Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(conv8)
    print("up9=",up9.shape)
    merge9 = concatenate([x,up9],axis = 3)
    print("merge9=[x,up9]",merge9.shape)
    conv9 = Conv2D(32, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)
    print("conv9", conv9.shape)
    conv10 = Conv2D(2, 3, activation='relu', padding='same')(conv9)
    print("conv10", conv10.shape)
    up10 =  Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(conv9)
    print("up10=",up10.shape)
    conv10 = Conv2D(16, 3, activation='relu', padding='same')(up10)
    conv10 = Conv2D(16, 3, activation='relu', padding='same')(conv10)
    conv10 =outputs = Conv2D(n_classes, (1, 1), activation='softmax')(conv10)
    print("conv10", conv10.shape)
    model = Model(inputs=inputs, outputs=conv10)
    return model

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_masks_reshaped_encoded),
                                                 train_masks_reshaped_encoded)
print("Class weights are...:", class_weights)

In [ ]:

IMG_HEIGHT = X1.shape[1]
IMG_WIDTH  = X1.shape[2]
IMG_CHANNELS = X1.shape[3]

def get_model():
    return unet_res(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

INIT_LR = 1e-5    # This value is specific to what model is chosen: Inception, VGG or ResNet etc.
EPOCHS = 50
#opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = get_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

#If starting with pre-trained weights. 
#model.load_weights('???.hdf5')

history = model.fit(X1, y_train_cat, 
                    batch_size = 1, 
                    verbose=1, 
                    epochs=EPOCHS, 
                    validation_data=(X_test, y_test_cat), 
                    class_weight=class_weights,
                    shuffle=False)

In [19]:
model.save('testOriginalImg.h5')

In [20]:
model = load_model('testOriginalImg.h5')

In [ ]:
acc,test_auc = model.evaluate(X_test, y_test_cat,batch_size=1)
print("Accuracy is = ", (acc * 100.0), "%")
print("test_auc is =", test_auc*100.0,"%")

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
auc = history.history['auccuracy']
train_auc = auc[49]
print("train_auc is =", train_auc*100.0,"%")

In [ ]:
train_mAP =  train_auc/3
test_mAP = test_auc/3
print("train_mAP is =", train_mAP*100.0,"%")
print("test_mAP is =", test_mAP*100.0,"%")

In [25]:
#model.load_weights('testOriginalImg.h5')  
#model.load_weights('sandstone_50_epochs_catXentropy_acc_with_weights.hdf5')  

#IOU
y_pred=model.predict(X_test,batch_size=1)
y_pred_argmax=np.argmax(y_pred, axis=3)

In [ ]:
#Using built in keras function
from keras.metrics import MeanIoU
n_classes = 3
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())


#To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
#class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0]+ values[2,0]+ values[3,0])
#class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1]+ values[2,1]+ values[3,1])
#class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2]+ values[1,2]+ values[3,2])
#class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[1,0]+ values[2,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] +  values[0,1]+ values[2,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1]  + values[0,2]+ values[1,2])
#class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
#print("IoU for class4 is: ", class4_IoU)

plt.imshow(train_images[0, :,:,0], cmap='gray')
plt.imshow(train_masks[0], cmap='gray')

In [ ]:
#import random
#test_img_number = random.randint(0, len(X_test)-1)
#print(test_img_number)
for test_img_number in range(len(X_test)):
    test_img = X_test[test_img_number]
    ground_truth=y_test[test_img_number]
    test_img_norm=test_img[:,:,0][:,:,None]
    test_img_input=np.expand_dims(test_img_norm, 0)
    prediction = (model.predict(test_img_input))
    predicted_img=np.argmax(prediction, axis=3)[0,:,:]


    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(test_img[:,:,0], cmap='gray')
    plt.subplot(232)
    plt.title('Testing Label')
    plt.imshow(ground_truth[:,:,0], cmap='jet')
    plt.subplot(233)
    plt.title('Prediction on test image')
    plt.imshow(predicted_img, cmap='jet')
    plt.show()